In [1]:
# Standard libraries
import re
import sys
import warnings
import pickle

# Data manipulation and visualization
import pandas as pd
import numpy as np

# Text processing
from sklearn.feature_extraction.text import CountVectorizer

# Machine learning models and metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings("ignore", category=FutureWarning)
sys.path.append("..")

# Local modules
from utilities.download_dataset import download_dataset
from utilities.update_model_metrics import update_model_metrics 
from utilities.test_process import text_process


In [2]:
GRID_SEARCH = False

In [3]:
download_dataset()

Downloading...
From (original): https://drive.google.com/uc?id=1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01
From (redirected): https://drive.google.com/uc?id=1atKYBZ9Pq3-zT0_BC_nZqMx0W-wjEW01&confirm=t&uuid=b9d551a1-7f09-4e02-a82b-14e65360754d
To: /Users/wojciech.neuman/Documents/ai-content-detectors/dataset/train.jsonl
100%|██████████| 292M/292M [00:03<00:00, 76.8MB/s] 


Downloading...
From: https://drive.google.com/uc?id=1FhT3m_ApKzX615JzshB5-d-j6S91-6oz
To: /Users/wojciech.neuman/Documents/ai-content-detectors/dataset/valid.jsonl
100%|██████████| 55.1M/55.1M [00:00<00:00, 68.7MB/s]


Downloading...
From: https://drive.google.com/uc?id=16p0td9GgJRb9AP8i4HlX-xZGI2u849uA
To: /Users/wojciech.neuman/Documents/ai-content-detectors/dataset/test.jsonl
100%|██████████| 39.1M/39.1M [00:00<00:00, 63.9MB/s]


All files have been downloaded and saved in the '../dataset' folder.


In [4]:
train_file_path = '../dataset/train.jsonl'
test_file_path = '../dataset/test.jsonl'

In [5]:
train_df = pd.read_json(train_file_path, lines=True)
test_df = pd.read_json(test_file_path, lines=True)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101284 entries, 0 to 101283
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   uid     101284 non-null  object
 1   text    101284 non-null  object
 2   extra   101284 non-null  object
 3   source  101284 non-null  object
 4   label   101284 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.9+ MB


In [7]:
train_df.head()

,uid,text,extra,source,label
0,[urlsf_subset00]-[15],The dangers of Illinois as a ‘right to work’ s...,"{'source': 'openweb', 'variant': 'original'}",openweb,0
1,[urlsf_subset00]-[15],"The governor of Illinois, Gov. Rauner, has req...","{'source': 'chatgpt', 'variant': 'original'}",chatgpt,1
2,[urlsf_subset00]-[83],Check current weather conditions\n\nIt’s going...,"{'source': 'openweb', 'variant': 'original'}",openweb,0
3,[urlsf_subset00]-[83],Check current weather conditions It’s going to...,"{'variant': 'original', 'source': 'llama'}",llama,1
4,[urlsf_subset00]-[89],"On Thursday, the president of the United State...","{'source': 'openweb', 'variant': 'original'}",openweb,0


In [8]:
train_df['text'] = train_df['text'].apply(text_process)
test_df['text'] = test_df['text'].apply(text_process)

In [9]:
X_train = train_df['text']
y_train = train_df['label']

X_test = test_df['text']
y_test = test_df['label']

In [10]:
# instantiate the vectorizer
vect = CountVectorizer()

# fit and transform X_train to create the document-term matrix
X_train_dtm = vect.fit_transform(X_train)

In [11]:
print(type(X_train_dtm), X_train_dtm.shape)

<class 'scipy.sparse._csr.csr_matrix'> (101284, 547607)


## Train model

#### Hyperparameter Optimization with GridSearchCV

In [12]:
best_params = {}

if GRID_SEARCH:
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.1, 1, 10, 100, 1000],
        'solver': ['lbfgs', 'liblinear', 'sag'],
        'max_iter': [200, 400, 800, 1600]
    }

    logreg = LogisticRegression()

    grid_search_lr = GridSearchCV(logreg, param_grid, cv=5, scoring='f1', n_jobs=-1, verbose=3)

    grid_search_lr.fit(X_train_dtm, y_train)

    best_params = grid_search_lr.best_params_

else: 
    best_params = {
        'penalty': 'l1',
        'C': 0.1,
        'solver': 'liblinear',
        'max_iter': 200,
    }

In [13]:
logreg = LogisticRegression(**best_params, random_state=42)

%time logreg.fit(X_train_dtm, y_train)

CPU times: user 5.43 s, sys: 259 ms, total: 5.69 s
Wall time: 5.87 s


LogisticRegression(C=0.1, max_iter=200, penalty='l1', random_state=42,
                   solver='liblinear')

## Test model

In [14]:
# transform X_test into a document-term matrix
X_test_dtm = vect.transform(X_test)

print(type(X_test_dtm), X_test_dtm.shape)

# make class predictions for X_test_dtm
y_pred = logreg.predict(X_test_dtm)

<class 'scipy.sparse._csr.csr_matrix'> (13952, 547607)


## Calculate metrics and update results file

In [15]:
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('F1 score: ', f1_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))

Accuracy:  0.8853211009174312
F1 score:  0.8874032371569317
Precision:  0.8715786563450373
Recall:  0.9038130733944955


In [16]:
test_df['pred_label'] = y_pred
test_df['correct_prediction'] = test_df['label'] == test_df['pred_label']

In [17]:
test_df.to_csv('../results/logistic_regression_test_results.csv', index=False)

In [18]:
all_results = pd.read_csv('../results/metrics_results.csv')

all_results = update_model_metrics(all_results, 'Logistic Regression', y_test, y_pred)


# Save the updated DataFrame back to the CSV file
all_results.to_csv('../metrics_results.csv', index=False)

all_results

,model,accuracy,precision,recall,f1_score
0,Logistic Regression,0.885,0.872,0.904,0.887
1,SVM,0.864,0.878,0.846,0.862
2,CNN,0.902,0.885,0.924,0.904
3,BiLSTM (RNN),0.935,0.954,0.914,0.934
4,BERT,0.963,0.952,0.974,0.963


## Save model

In [19]:
filename = '../saved_models/logistic_regression_model.pkl'
pickle.dump(logreg, open(filename, 'wb'))